In [1]:
from component import tile 
from component import model

Styles()

ResizeTrigger()

In [2]:
# # for debug 
# import pandas as pd 
# import geopandas as gpd
# from component import parameter as cp

# tb_model = model.TableModel()
# tb_model.json_table = '{"pathname": "/home/prambaud/modules/clip-time-series/utils/clip_test_points.csv", "id_column": "id", "lat_column": "Y", "lng_column": "X"}'


# df = pd.read_csv(cp.test_dataset, sep=None, engine='python')
# df = df.filter(items=["lat", "lng", "id"])
# gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lng, df.lat), crs='EPSG:4326')
# tb_model.pts = gdf 

In [3]:
viz_model = model.VizModel()

In [4]:
viz_input_tile = tile.InputTile(viz_model, tb_model)

In [5]:
viz_input_tile

InputTile(align_center=True, children=[Card(children=[Html(children=['set up your inputs'], tag='h2'), Flex(ch…